In [15]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 80)
from shapely.geometry import Point, LineString
from geopandas import GeoDataFrame
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

## Classifying investor purchases

Process to classify deeds as (a) residential properties, (b) likely purchased by investors.

## 1. Selecting deeds that fall within residential parcels

Using land use data from the city of Philadelphia (see original source [here](https://www.opendataphilly.org/dataset/land-use))

In [4]:
# read in land use file
landuse = gpd.read_file("data/Land_Use_2019/Land_Use.shp")

In [7]:
# creating a more narrow set of residential parcels
# just low and medium residential uses
res = landuse.loc[(landuse['C_DIG2DESC']=='Residential Medium') | 
                    (landuse['C_DIG2DESC']=='Residential Low')]

# dropping pacels with na values for their geometry
res = res.dropna(subset=['geometry'])

# reprojecting residential low/med only category
res = res.to_crs('epsg:2272')

In [16]:
# reading in the real estate tax transfer data
rtt = pd.read_csv("data/rtt_summary.csv")

/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (26,34,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
# selecting just deeds (not mortgages and other transfer taxes)
deeds = rtt.loc[rtt['document_type'] == "DEED"]
# adding a year column from the document date
deeds["sort_year"] = deeds['display_date'].astype(str).str[:4]

/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [64]:
# defining points from lat/lon
deeds_points = [Point(xy) for xy in zip(deeds.lat,deeds.lng)]
deedsgeo = GeoDataFrame(deeds, geometry=deeds_points)

In [85]:
# selecting only deeds with non zero state taxes
deedsgeo = deedsgeo.loc[deedsgeo["state_tax_amount"] > 0]

# and that also have an address field
deedsgeo= deedsgeo.loc[deedsgeo["address_low"]>0]

In [71]:
# setting the projection for the lat lon coordinates 
deedsgeo.crs = 'epsg:4326'

In [74]:
# reprojecting to: EPSG:2272: NAD83 / Pennsylvania South (ftUS)
deedsgeo = deedsgeo.to_crs('epsg:2272')

In [78]:
# spatial join between deeds and residential parcels (to find only those deeds within residential parcels)
deeds_res = gpd.sjoin(deedsgeo,res, how="inner",op="within")

## 2. Identify likely investor purchases
First tag all grantees with corporate entity names. Then find all investors who purchased more than 2 properties in any one year span.   

### 2A. Identifying Deeds with likely investors as the grantee based on grantee name
First tag all investor named entities   
  - LLC
  - LP
  - LIMITED PARTNERSHIP
  - CORP
  - CORPORATION
  - INC
  - COMPANY
  - LIMITED PARTNE
  - L P
  - L L C
  - PARTNERSHIP
  - LTD
  - TRUST 
    


In [87]:
# selecting just post-2000 deeds
deeds_res['sort_year'] = pd.to_numeric(deeds_res['sort_year'])
deeds_res = deeds_res.loc[deeds_res['sort_year']>1999]

# selecting investors by grantee name
inv = deeds_res['grantees'].str.contains(
    'LLC|LP|LIMITED PARTNERSHIP|CORP|CORPORATION|INC|COMPANY|LIMITED PARTNE|L P|L L C|PARTNERSHIP|LTD|TRUST'
    ,regex=True)

# converting investors series to dataframe
inv = inv.to_frame()

# merging with deeds
deeds_inv = deeds_res.merge(
    inv, left_index=True, right_index = True)
deeds_inv = deeds_inv.dropna(subset=['grantees_x'])

# creating a count column to allow grouping and count 
# for investor and owner by year 
deeds_inv['inv'] = deeds_inv['grantees_y'].apply(lambda x: 1 if x == True else 0)
deeds_inv['occ'] = deeds_inv['grantees_y'].apply(lambda x: 1 if x == False else 0)

# dropping duplicate object ids from the dataset
deeds_inv = deeds_inv.drop_duplicates(subset="objectid")

### 2B. Classifying investor purchases by multiple purchasers
Identifying grantees who purchased more than 2 properties in any one year period from the full set of non-investor classified deeds:

In [91]:
# finding multiple purchasers from the full set of non-investor classified deeds
# selecting from just the non llc llp deeds
occ = deeds_inv.loc[deeds_inv['occ']==1]

In [92]:
# getting all deeds from purchasers who are associated with more than one deed
multiple_purchasers = occ.groupby(
    "grantees_x").filter(lambda s: s['objectid'].count()>=2)


In [93]:
# converting date to datetime format
multiple_purchasers['display_date'] = multiple_purchasers[
    'display_date'].astype('datetime64[ns]')

In [94]:
# dropping unneeded columns
columns_multi = ['objectid','grantors','grantees_x',
                 'display_date','sort_year','street_address']
multiple_purchase_simple = multiple_purchasers[columns_multi]


In [95]:

multiple_purchase_simple[
    'elapsed'] = multiple_purchase_simple.sort_values(
    ['grantees_x','display_date']).groupby(
    'grantees_x')['display_date'].diff()

# converting timedelta column to an int type 
# to make it possible to filter below
multiple_purchase_simple[
    'elapsed_days'] = multiple_purchase_simple[
    'elapsed'].dt.days


/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [98]:
# filtering for deeds from grantees who purchased two properties within 1 year of each other
multiple_investors = multiple_purchase_simple.groupby('grantees_x').filter(
    lambda x: x['elapsed_days'].min() <= 365)
multiple_investors['multi_invest'] = 1

## 4. merging multiple purchase investors back with full set of deeds

Compiling a comprehensive classification of investor purchases 

In [128]:
deeds_multi = deeds_inv.merge(
    multiple_investors, left_on="objectid", right_on="objectid",how="left")

deeds_multi["investor_all"] = deeds_multi[
    "inv"] + deeds_multi["multi_invest"].fillna(0)
deeds_multi['owner_occup'] = deeds_multi[
    'investor_all'].apply(lambda x: 0 if x == 1 else 1)

In [131]:
# Saving classified deeds to speed up future processing time 
deeds_for_save = deeds_multi
deeds_for_save['elapsed'] = deeds_for_save['elapsed'].fillna(pd.Timedelta('999 days'))
deeds_for_save['elapsed'] = deeds_for_save['elapsed'].dt.days.astype('int16')
deeds_for_save['display_date_y'] = deeds_for_save['display_date_y'].astype(str)


In [132]:
# saving final list of grantees
with open("data/rtt_investors_classified.geojson",
          'w') as f:
    f.write(deeds_for_save.to_json())

## 5. Reimport geojson of classified deeds

To reimport classified deeds use gpd.read_file and then set the coordinate reference system:

In [133]:
# read in classified deeds file
rtt_classified = gpd.read_file("data/rtt_investors_classified.geojson")

In [143]:
# Specify coordinate reference system
rtt_classified.crs = 'epsg:2272'